University of Michigan - ROB 101 Computational Linear Algebra

# Homework 4: Building Your Own Functions for Solving Equations


### Up to now, we have been wrting "scripts'', that is, a series of commands that allow us to accomplish a goal. Scripts are are great tool and a super quick way to prototype ideas. We are now going to move on to writing "functions'', which allow us to call our operations in a much easier manner. 

### Another thing we'll do is some error checking. Once you have hidden your code behind a function, someone else might try to use it in a manner that will throw bugs or much worse, produce erroneous results without throwing a bug and hence without warning the user. The funny part is, that "user'' is often "us''! We build a function and then we forget exactly all of the special requirements on the data that the function uses to produce an answer! 

### Example: Forward Substitution

In [1]:
#computes the solution x to a system Lx = b where L is a lower triangular matrix and b is a column vector
#note the structure we have used: we being by declaring "function" and we end by declaring "end". On 
# the line befor "end", we specify that value or values we are to "return".
#
# Everything in between is operating on the data passed to the function.
function forwardsub(L, b)
    # START of our computations
    n = length(b)
    x = Vector{Float64}(undef, n); #initialize an x vector of the correct size
    x[1] = b[1]/L[1,1] #find the first entry of x
    for i = 2:n #find every entry from the 2nd to the end
        x[i]=(b[i]- (L[i,1:i-1])' *x[1:i-1] )./L[i,i] 
        #notice that we used a transpose operator to get the row of L
    end
    # END of our computations. Everything between START and END
    # is essentailly the same as a script. It's that easy to build a function!
    return x
end
            

forwardsub (generic function with 1 method)

In [2]:
#Here is an example where the function works!
L = [1 0 0 ; 2 3 0; 4 5 6]
b = [1; 8; 32]
forwardsub(L, b)

3-element Vector{Float64}:
 1.0
 2.0
 3.0

### Problem 1.  


In [3]:
#Here is one way that we can break our function:
L = [6 0 0 ; 2 1 0; 8 5 0]
b = [3; 2; 3]
forwardsub(L, b)

3-element Vector{Float64}:
   0.5
   1.0
 -Inf

#### a) Run the cell above, then explain, in a sentence, why the error occurred
**Write your response here:**


Hint: It may help to write out the matrix L, then walk through the steps of the function

In [ ]:
The error occurred because there is no value in the third row of the third column

In [4]:
#Here is another way that it could break:
L = [5 0 0 ; 1 4 0; 7 3 2]
b = [3; 2; 3; 4]
forwardsub(L, b)

LoadError: BoundsError: attempt to access 3×3 Matrix{Int64} at index [4, 1:3]

#### b) Run the cell above and explain why the error occurred
**Write your response here:**




Hint: Write the system in regular(non-matrix) notation, we will do the first equation for you:
$$5x_1 + 0x_2 + 0x_3 = 3$$  you do the rest!

In [ ]:
There was an error because vector b had too many rows so the number of rows didnt match between L and b.

In [6]:
#The above cases produced errors, so we knew to look for a problem in the data. 
#This example will not produce an error, but it also doesn't produce the solution we expect
using Random 
Random.seed!(4321) 
L=rand(3,3) 
b=rand(3,1) 
x=forwardsub(L, b)
println("Oops!")
println(L)
println(b)

solutionError=L*x-b 

Oops!
[0.3726845628953891 0.2984037406608038 0.7677873754911297; 0.34774159399886917 0.9291884995571108 0.028949078446419585; 0.3431192612171122 0.517511261534207 0.44639680260927006]
[0.570463189056158; 0.33978952044375144; 0.42204825906111765;;]


3×1 Matrix{Float64}:
 -0.05485307024652819
  0.0002626216008894122
  0.0

#### c) Run the cell above and explain why x does not satisfy Lx = b
**Write your response here:**


Hint:  You may want to add additional print statements in order to see the matrices you are using.

In [ ]:
We didn't get the result we expected because each value in the matrix and vector was a long decimal and not a whole number.

### Error Checking:  An example
Above, you experienced three possible ways that this function can fail.  Let's build a few error checks into the function so that we can tell what is going wrong.  Follow along with the comments in the code below to see how we can do these checks.

Now, if you go back and run the erroneous cells from above, the function will print out a more helpful error.

In [7]:
#I copied the forwardsub function from above
function forwardsub(L, b)
    # Now before we start our computations, let's make sure that there will be no errors
    n = length(b)
    
    #first, let's check that our inputs are the right size
    (rows, cols) = size(L)
    if rows != cols
        print("L is not a square")
        return 0
    end
    if rows != n
        print("L and b are not size compatible")
        return 0
    end
    #if we got to here, that means we have a matrix and vector of compatible sizes
    
    #now, let's check that L is actually lower triangular, eg: every element above the diagonal is zero
    #also check to make sure that there are no zeros on the diagonal
    for i in 1:n
        for j in 1:n
            if j > i
                if L[i, j] != 0
                    print("L is not lower triangular")
                    return 0
                end
            elseif j == i
                if L[i, j] == 0
                    print("There is a zero on the diagonal")
                    return 0
                end
            end
        end
    end
                    
    
    #Now we can actually do the forward substitution, now that we found the arguments to be acceptable
    
    x = Vector{Float64}(undef, n); #initialize an x vector of the correct size
    x[1] = b[1]/L[1,1] #find the first entry of x
    for i = 2:n #find every entry from the 2nd to the end
        x[i, 1] = (b[i]- (L[i,1:i-1])' *x[1:i-1] )./L[i,i] 
        #notice that we used a transpose(') operator to get the row of L
    end
    # END of our computations. Everything between START and END
    # is essentailly the same as a script. It's that easy to build a function!
    return x
end

forwardsub (generic function with 1 method)

### Problem 2.  Write a function for back substitution.
Use the function given to you in problem 1, and section 3.5 of the ROB 101 booklet to help you construct your answer.  Think about just doing forward substitution, but backwards.  

Hint:  In order to make a for loop that goes backwards, you can either use

**for i in reverse(1:n)**

or

**for i in n:-1:1**  (where the second parameter(-1) is the step size)

In [8]:
#computes the solution x to a system Ux = b where U is a lower triangular matrix and b is a column vector
function backwardsub(U, b)
    n = length(b)
    x = Vector{Float64}(undef, n) #initialize an x vector of the correct size
    
    #fill in the rest of the function here.  You should think about what the forward substitution algorithm looked like
### BEGIN SOLUTION
    x[n] = b[n]/U[n,n]
    for i in n-1:-1:1
        x[i]=(b[i]- (U[i,(i+1):n])' *x[(i+1):n] )./U[i,i]
    end
    return x
### END SOLUTION
    
    
end
    

backwardsub (generic function with 1 method)

In [12]:
##Unit test
U = [4 5 6; 0 2 3; 0 0 1 ]
b = [32; 13; 3]
answer2a = backwardsub(U, b)
### BEGIN HIDDEN TESTS
#@assert answer2a == [1;2;3]
### END HIDDEN TESTS

3-element Vector{Float64}:
 1.0
 2.0
 3.0

### Problem 3.  Add Error testing, like the example from Problem 1, to your function for backwardsub
Copy your code from problem 2, and make sure that you are checking for these errors:
* Check that the matrix U is a square and is compatible with the size of b
* Check that the matrix U is upper triangular
* Check that there are no zeros on the diagonal of U

In [13]:
#copy your code from above, and add error checks

###BEGIN SOLUTION
function backwardsub(U, b)
    n = length(b)
    
    #checking for size compatibility
    (rows, cols) = size(U)
    if rows != cols
        print("U is not a square")
        return 0
    end
    if rows != n
        print("U and b are not size compatible")
        return 0
    end
    
    #checking for upper triangular
     for i in 1:n
        for j in 1:n
            if  i > j
                if U[i, j] != 0
                    print("U is not upper triangular")
                    return 0
                end
            elseif j == i
                if U[i, j] == 0
                    print("There is a zero on the diagonal")
                    return 0
                end
            end
        end
    end
    
    #actual solving
    x = Vector{Float64}(undef, n) #initialize an x vector of the correct size
    x[n] = b[n]/U[n,n]
    for i in n-1:-1:1
        x[i]=(b[i]- (U[i,(i+1):n])' *x[(i+1):n] )./U[i,i]
    end
    return x
end
### END SOLUTION

backwardsub (generic function with 1 method)

In [14]:
#The function should work here.  There should not be any error message
U1 = [1 2 3; 0 4 5; 0 0 6]
b1 = [14; 23; 18]
backwardsub(U1, b1)

3-element Vector{Float64}:
 1.0
 2.0
 3.0

In [15]:
#You should have an error message about a zero on a diagonal
U2 = [6 8 1 ; 0 0 3; 0 0 4]
b2 = [3; 2; 3]
backwardsub(U2, b2)

There is a zero on the diagonal

0

In [16]:
#You should have an error message that U is not upper triangular
U3 = [2 8 1 9; 3 0 3 9; 0 0 4 2; 0 0 0 1]
b3 = [3; 2; 3; 1]
backwardsub(U3, b3)

U is not upper triangular

0

In [17]:
#These matrices are not size compatible, your error should reflect that
U4 = [5 4 1; 6 7 2; -7 2 1]
b4 = [5; 1; 2; 6]
backwardsub(U4, b4)

U and b are not size compatible

0

### Problem 4.  LU decomposition with Permutations
Julia has a built-in LU Factorization function. Below is an example of how to access the properties of the lu() function.  

In [18]:
using LinearAlgebra
M = [0 4 2; 10 2 1; 1 1 1 ]
# F stands for Factorization. It contains all three matrices, L,U, and P
F = lu(M)
L =  F.L #lower triangular factor
@show L
U =  F.U #upper triangular factor
@show U
P = F.P #Permutation matrix
@show P
##properties of the factorization and a check that we know how to use it!
Check = P*M-L*U

L = [1.0 0.0 0.0; 0.0 1.0 0.0; 0.1 0.2 1.0]
U = [10.0 2.0 1.0; 0.0 4.0 2.0; 0.0 0.0 0.5]
P = [0.0 1.0 0.0; 1.0 0.0 0.0; 0.0 0.0 1.0]


3×3 Matrix{Float64}:
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

#### a) Return the Lower triangular matrix of the LU decomposition of matrix G
$$ G = \begin{bmatrix}6&7&2&9\\1&-3&-5&6\\-8&2&-3&-4\\0&0&2&1\end{bmatrix} $$

In [19]:
#Return your answer as a variable named answer3a
#replace the lines below with your code

### BEGIN SOLUTION
G = [6 7 2 -9; 1 -3 -5 6; -8 2 -3 -4; 0 0 2 1]
answer4a = lu(G).L
### END SOLUTION

4×4 Matrix{Float64}:
  1.0     0.0        0.0       0.0
 -0.75    1.0        0.0       0.0
 -0.125  -0.323529   1.0       0.0
 -0.0     0.0       -0.366577  1.0

#### b) Return the Permutation matrix and the Upper triangular matrix of matrix C
$$ C = \begin{bmatrix}6&-2&8&-7&1\\-2&0&3&5&9\\9&-4&5&-1&0\\-8&2&-2&3&1\\6&0&-5&1&9\end{bmatrix}$$

In [20]:
#return the permutation matrix as perm2b and the upper triangular matrix as upper2b

### BEGIN SOLUTION
C = [6 -2 8 -7 1; -2 0 3 5 9; 9 -4 5 -1 0; -8 2 -2 3 1; 6 0 -5 1 9]
perm4b = lu(C).P
upper4b = lu(C).U
### END SOLUTION

5×5 Matrix{Float64}:
 9.0  -4.0       5.0      -1.0       0.0
 0.0   2.66667  -8.33333   1.66667   9.0
 0.0   0.0       6.75     -6.75     -1.25
 0.0   0.0       0.0       6.66667  12.2469
 0.0   0.0       0.0       0.0       4.57778

### Problem 5.  Use LU decompostition and forward/back substitution to solve the system of equations for x.
$$\begin{bmatrix}8&-18&-16&2 &0& -8& 18& 8& 12 &-20\\ -36& 41& 142 &21& -20 &6 -111& -106& -24& 190\\
    36 &-117 &-13& 38 &-25 &-60 &63& -31& 79 &9\\ 32& -104& 32& 44& 26& -114 &-82& 52& 92& -130\\
    24& -78& -10 &26& -46 &-30 &36 &-10& 70& -15\\ 12 &-19 &-70 &-43 &-48 &142& 190 &-7 &38 &64\\
    16& -52& -40& 26& -136& 80& 126& 1& 69 &39\\ 20 &-9 &-63 &-66& 190& -129& 49& -10& -129& -199\\
    -28& 51 &41 &-4 &-3 &22 &119 &-261 &-143 &383\\ 36 &-61 &-123 &10 &-88 &-55 &1 &113 &95 &-173\end{bmatrix} x = \begin{bmatrix}-1\\2\\-2\\8\\-9\\-7\\-7\\-2\\-1\\-5\end{bmatrix} $$

Hint: For a system $Ax = b$, if the LU decompositon without permutations is performed on A, then $LUx = b$, $Ux = y$, and $Ly = b$.  See page 49 of the ROB 101 course booklet for an example.

In [21]:
#Solve the problem with julia's lu() function and your forward and back substitution functions 
#Matrices A and b are declared for you
A = [8 -18 -16 2 0 -8 18 8 12 -20; -36 41 142 21 -20 6 -111 -106 -24 190;
    36 -117 -13 38 -25 -60 63 -31 79 9; 32 -104 32 44 26 -114 -82 52 92 -130;
    24 -78 -10 26 -46 -30 36 -10 70 -15; 12 -19 -70 -43 -48 142 190 -7 38 64;
    16 -52 -40 26 -136 80 126 1 69 39; 20 -9 -63 -66 190 -129 49 -10 -129 -199;
    -28 51 41 -4 -3 22 119 -261 -143 383; 36 -61 -123 10 -88 -55 1 113 95 -173]
b = [-1;2; -2; 8; -9; -7; -7; -2; -1; -5]

10-element Vector{Int64}:
 -1
  2
 -2
  8
 -9
 -7
 -7
 -2
 -1
 -5

In [22]:
#Remember to use lu(M, Val(false)) in order to prevent permutations
using LinearAlgebra
### BEGIN SOLUTION
L,U = lu(A, Val(false))
#now solving Ly = b
y = forwardsub(L, b)
#now solving Ux = y
x = backwardsub(U, y)
### END SOLUTION

10-element Vector{Float64}:
  122.99797869786539
  214.12066452749986
  156.8752795269436
  -81.54624463683945
  -35.6625467798738
 -186.40406197012416
  155.88988965335534
  119.30768163675955
   91.14635130887423
   40.15915423525528

In [23]:
### BEGIN HIDDEN TESTS
invSol = inv(A) * b
@assert invSol == x
### END HIDDEN TESTS

LoadError: AssertionError: invSol == x